In [3]:
import zipfile
import shutil
import numpy as np
import matplotlib.pyplot as plt
# sphinx_gallery_thumbnail_path = '_static/playback-thumbnail.png'

import librosa
# We'll need IPython.display's Audio widget
from IPython.display import Audio

# We'll also use `mir_eval` to synthesize a signal for us
import mir_eval.sonify

import csv
import os
import json



#TODO: extract only comp audio files

In [34]:
def unzip_file(data_path, zipped_name):
    '''
    Unzip specific file inside of aux_folder
    '''
    with zipfile.ZipFile(data_path) as zf:
        zf.extract(zipped_name, 'aux_folder/')
        #zf.extract('dir_sub/file_sub.txt', 'dir_out_extract')


def name_jams(audio_name):
    '''
    Get the name of audio metatada inside of annotation.zip
    '''
    jams_name = audio_name.replace('_hex', '').replace('wav', 'jams')
    return jams_name


def remove_solos(files_list):
    '''
    Remove solo files from list of files
    '''
    res = []
    for i in files_list:
        if 'solo' in i:
            pass
        else:
            res.append(i)
    
    return res


In [95]:


def read_metadata(metadata_name):
    # Opening JSON file
    f = open('aux_folder/' + metadata_name)
    
    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    chord = data['annotations'][14]['data']
    #key_mode = data['annotations'][16]['data']

    return chord




#ref -> https://thepythoncode.com/assistant/transformation-details/cutting-audio-files-in-python-with-pydub/
from pydub import AudioSegment

def cut_audio(input_file, output_file, start_time, end_time):
    """
    Cut parts of an audio file and save it to a new file.

    Args:
        input_file (str): Path to the input audio file.
        output_file (str): Path to save the output audio file.
        start_time (int): Start time in milliseconds.
        end_time (int): End time in milliseconds.
    """
    # Load the audio file
    audio = AudioSegment.from_file(input_file)

    # Cut the audio
    cut_audio = audio[start_time:end_time]

    # Save the cut audio to a new file
    cut_audio.export(output_file, format="wav")





def cut_all(chord_dict, audio_path, output_path, og_file, batch):
    '''
    chord_dict: dict, contendo todos os acordes do áudio original
    audio_path: str, caminho para áudio original
    output_path: str, pasta onde os arquivo cortados serão salvos
    og_file: str, nome do arquivo original para documentação
    '''
    iter = np.random.randint(0,999)
    for i in chord_dict:

        start = i['time']*1000
        end = (i['time'] + i['duration'])*1000
        value = i['value']
        filename = ('file_' + str(batch) + str(iter) + '.wav').replace(':', '')
        output = (output_path + filename)
        
        cut_audio(audio_path, output, start, end)
        # Documenta metadados do arquivo cortado
        with open('metadata.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([filename, output_path, value, og_file])
        
        iter +=1




In [93]:
def preprocess_audio():
    # zip file handler  
    data_path = 'audio_hex-pickup_original.zip'
    annotations_path = 'annotation.zip'

    aux_folder = 'aux_folder/'
    prepro_folder = 'processed_audio/'


    zip = zipfile.ZipFile(data_path)
    # arquivos disponíveis no arquivo zipado
    zipped_files = zip.namelist()

    # remove versões de solo 
    zipped_files = remove_solos(zipped_files)
    total_chords = 0
    batch = 1
    for i in zipped_files:
        metadata_name = name_jams(i)
        

        # Unzipa o arquivo i dentro da pasta aux_folder
        unzip_file(data_path, i)
        # Unzipa o arquivo .jams de metadados do arquivo i dentro da pasta aux_folder
        unzip_file(annotations_path, metadata_name)

        # Recupera o dicionário que marca a duração de cada acorde
        chord = read_metadata(metadata_name)
        
        
        cut_all(chord, aux_folder + i, prepro_folder, i, batch)

        total_chords += len(chord)
        os.remove(aux_folder + i)
        os.remove(aux_folder + metadata_name)


        batch +=1
    
    return f'{total_chords} chords were (hopefully!) extracted'

In [97]:
preprocess_audio()

'2160 chords were (hopefully!) extracted'

In [48]:
# hear the audio
y, sr = librosa.load('aux_folder/' + audio_name)

Audio(data=y, rate=sr)

In [119]:
audio